## Simple mapper

Given the following source called 'MyFile':

| id | first_name | phone |
| - | - | - |
| 1  | Buffy | 555-555-5555 |

Then the target 'Catalyst Contact' should match the example:

| Student_ID__c | FirstName | Phone |
| - | - | - |
| 1  | Buffy | 555-555-5555 |

## Simple transform

Given the following source called 'MyFile':

| id | name |
| - | - |
| 1  | Summers, Buffy |

Then the target 'Catalyst Contact' should match the example:

| Student_ID__c | FirstName | LastName |
| - | - | - | - |
| SDH1  | Buffy | Summers |


## Deduper

Given the following source called 'MyFile':

| id | name |
| - | - |
| 1  | Buffy |
| 1  | Faith |

Then the target 'Catalyst Contact' should match the example:

| Student_ID__c | FirstName |
| - | - | - |
| SDH1  | Buffy |


## Join

Given the following source called 'MyFile':

| id | name |
| - | - |
| 1  | Buffy |

And the following source called 'ClassesFile':

| id | student_id | name |
| - | - | - |
| 1 | 1 | Psychology |
| 2 | 1 | History |
| 3 | 1 | Physiology |

Then the target 'Catalyst Contact' should match the example:

| Class_Record_ID__c | FirstName | ClassName |
| - | - | - |
| SDH1-1  | Buffy | Psychology |
| SDH1-2  | Buffy | History |
| SDH1-3  | Buffy | Physiology |
